In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
from itertools import chain

In [2]:
data_dir = '../data/'

# Replogle K562 cells

In [3]:
# load data with annotated perturbations
data = sc.read_h5ad(data_dir + 'Replogle/K562_essential_raw_singlecell_01.h5ad')

In [56]:
# calculate the number of cells for each gRNA pair
gRNA_frequencies = pd.DataFrame({'counts': data.obs['gene_transcript'].value_counts()})
targeting_gRNAs = gRNA_frequencies[~gRNA_frequencies.index.str.contains('non-targeting')]
nt_gRNAs = gRNA_frequencies[gRNA_frequencies.index.str.contains('non-targeting')]

# Calculate quantiles
quantiles_t = np.percentile(targeting_gRNAs['counts'], [0, 25, 50, 75, 100])
quantiles_nt = np.percentile(nt_gRNAs['counts'], [0, 25, 50, 75, 100])

# Select random rows from each quantile
selected_gRNAs = pd.DataFrame()

for i in range(len(quantiles_t) - 1):
    # targeting gRNAs
    quantile_range = (targeting_gRNAs['counts'] >= quantiles_t[i]) & (targeting_gRNAs['counts'] < quantiles_t[i + 1])
    quantile_rows = targeting_gRNAs[quantile_range].sample(n=10, replace=True)
    selected_gRNAs = pd.concat([selected_gRNAs, quantile_rows])
    
    # non-targeting gRNAs
    quantile_range = (nt_gRNAs['counts'] >= quantiles_nt[i]) & (nt_gRNAs['counts'] < quantiles_nt[i + 1])
    quantile_rows = nt_gRNAs[quantile_range].sample(n=3, replace=False)
    selected_gRNAs = pd.concat([selected_gRNAs, quantile_rows])
    
selected_gRNAs = selected_gRNAs.reset_index().rename(columns={'index': 'gRNA_pair_id'})

In [57]:
# select all gRNAs belonging to the selected gRNA pairs
gRNA_target_df = pd.read_csv(data_dir + 'Replogle/K562_essential_processed_annotation.csv')
selection = selected_gRNAs.merge(gRNA_target_df)

# save the results
selection.to_csv(data_dir + 'Replogle/gRNA_pair_selection_50.csv')

In [58]:
selection.head()

,gRNA_pair_id,counts,gRNA,target_gene,ensembl_id,target_seq,full_gRNA
0,2579_EIF5_P2_ENSG00000100664,62,EIF5_-_103803091.23-P2,EIF5,ENSG00000100664,GATGACCGTTACATTGTCAA,EIF5_-_103803091.23-P2_posA
1,2579_EIF5_P2_ENSG00000100664,62,EIF5_+_103803074.23-P2,EIF5,ENSG00000100664,GAGATCCATTGACAATGTAA,EIF5_+_103803074.23-P2_posB
2,4944_MED21_P1P2_ENSG00000152944,63,MED21_-_27175714.23-P1P2,MED21,ENSG00000152944,GGGCGCCACCGCGAACTCGG,MED21_-_27175714.23-P1P2_posA
3,4944_MED21_P1P2_ENSG00000152944,63,MED21_-_27175691.23-P1P2,MED21,ENSG00000152944,GCGGCATAAAGCGCTCTCTC,MED21_-_27175691.23-P1P2_posB
4,490_ARID3A_P1P2_ENSG00000116017,31,ARID3A_+_926082.23-P1P2,ARID3A,ENSG00000116017,GGCTCCCGGCCCCCGACCCA,ARID3A_+_926082.23-P1P2_posA


In [61]:
list(pd.read_csv(data_dir+ 'Replogle/gRNA_pair_selection_50.csv')['gRNA'])

['EIF5_-_103803091.23-P2',
 'EIF5_+_103803074.23-P2',
 'MED21_-_27175714.23-P1P2',
 'MED21_-_27175691.23-P1P2',
 'ARID3A_+_926082.23-P1P2',
 'ARID3A_-_926025.23-P1P2',
 'TAF1D_-_93471390.23-P2',
 'TAF1D_+_93471338.23-P2',
 'TIMM44_-_8008522.23-P1P2',
 'TIMM44_+_8008475.23-P1P2',
 'EMC3_+_10052774.23-P2',
 'EMC3_+_10052758.23-P2',
 'RBM19_+_114404081.23-P1P2',
 'RBM19_+_114404111.23-P1P2',
 'RAB18_-_27793285.23-P1P2',
 'RAB18_-_27793292.23-P1P2',
 'INTS4_-_77705700.23-P1P2',
 'INTS4_+_77705685.23-P1P2',
 'GTF2E2_-_30515502.23-P1P2',
 'GTF2E2_+_30515555.23-P1P2',
 'non-targeting_02787',
 'non-targeting_01465',
 'non-targeting_00635',
 'non-targeting_02061',
 'non-targeting_02053',
 'non-targeting_02584',
 'MCM4_-_48873574.23-P1P2',
 'MCM4_+_48873564.23-P1P2',
 'RBBP6_+_24550948.23-P1P2',
 'RBBP6_+_24550884.23-P1P2',
 'MIPEP_+_24463500.23-P1P2',
 'MIPEP_+_24463484.23-P1P2',
 'TSR1_+_2239717.23-P1',
 'TSR1_+_2239506.23-P1',
 'UBAP1_-_34179122.23-P1P2',
 'UBAP1_+_34179139.23-P1P2',
 'EBNA1B

In [15]:
# add all non-targeting gRNAs to the selection
selected_gRNAs = set(list(pd.read_csv(data_dir+ 'Replogle/gRNA_pair_selection_50_old.csv')['gRNA_pair_id']) +
    list(nt_gRNAs.index))
gRNA_target_df = pd.read_csv(data_dir + 'Replogle/K562_essential_processed_annotation.csv')
selection = gRNA_target_df[gRNA_target_df['gRNA_pair_id'].isin(selected_gRNAs)]
selection.to_csv(data_dir + 'Replogle/gRNA_pair_selection_50.csv')

In [18]:
# save a renamed version to use for downstream analyses with SCEPTRE
selection = pd.read_csv(data_dir + 'Replogle/gRNA_pair_selection_50.csv')
selection = selection[['full_gRNA', 'gRNA_pair_id', 'target_gene']].drop_duplicates()
selection = selection.rename(columns={'full_gRNA': 'individual_gRNA', 'gRNA_pair_id': 'gRNA'})
selection.to_csv(data_dir + 'Replogle/gRNA_pair_selection_50_sceptre.csv', index = False)

# Replogle RPE1 cells

In [36]:
# also select 40 targeting gRNAs for RPE1 cells
data = sc.read_h5ad(data_dir + 'Replogle/rpe1_raw_singlecell_01.h5ad')

# calculate the number of cells for each gRNA pair
gRNA_frequencies = pd.DataFrame({'counts': data.obs['gene_transcript'].value_counts()})
targeting_gRNAs = gRNA_frequencies[~gRNA_frequencies.index.str.contains('non-targeting')]
nt_gRNAs = gRNA_frequencies[gRNA_frequencies.index.str.contains('non-targeting')]

# Calculate quantiles
quantiles_t = np.percentile(targeting_gRNAs['counts'], [0, 25, 50, 75, 100])

# Select random rows from each quantile
selected_gRNAs = pd.DataFrame()

for i in range(len(quantiles_t) - 1):
    # targeting gRNAs
    quantile_range = (targeting_gRNAs['counts'] >= quantiles_t[i]) & (targeting_gRNAs['counts'] < quantiles_t[i + 1])
    quantile_rows = targeting_gRNAs[quantile_range].sample(n=10, replace=True)
    selected_gRNAs = pd.concat([selected_gRNAs, quantile_rows])
    
selected_gRNAs = selected_gRNAs.index.tolist() +  nt_gRNAs.index.tolist()

# select all gRNAs belonging to the selected gRNA pairs
gRNA_target_df = pd.read_csv(data_dir + 'Replogle/RPE1_gRNA_target_df.csv')
selection = gRNA_target_df[gRNA_target_df['gRNA'].isin(selected_gRNAs)]
selection.to_csv(data_dir + 'Replogle/RPE1_gRNA_selection.csv', index = False)

In [37]:
selection

,individual_gRNA,gRNA,target_gene
57,ANAPC15_+_71823713.23-P1P2_posA,304_ANAPC15_P1P2_ENSG00000110200,ANAPC15
88,ARMC6_+_19144442.23-P1P2_posA,523_ARMC6_P1P2_ENSG00000105676,ARMC6
110,ATP5A1_-_43678159.23-P1_posA,653_ATP5F1A_P1_ENSG00000152234,ATP5F1A
162,BRD2_+_32940426.23-P1_posA,888_BRD2_P1_ENSG00000204256,BRD2
183,C14orf166_+_52456301.23-P1P2_posA,976_RTRAF_P1P2_ENSG00000087302,RTRAF
...,...,...,...
5341,non-targeting_02934_posB,11315_non-targeting_non-targeting_non-targeting,non-targeting
5342,non-targeting_02863_posB,11321_non-targeting_non-targeting_non-targeting,non-targeting
5343,non-targeting_03577_posB,11322_non-targeting_non-targeting_non-targeting,non-targeting
5344,non-targeting_00758_posB,11324_non-targeting_non-targeting_non-targeting,non-targeting
